In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

'c:\\Users\\jaesc2\\GitHub\\skforecast'

In [2]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries

In [3]:
from typing import Union, Tuple, Optional, Callable
import numpy as np
import pandas as pd
import warnings
import logging
from copy import deepcopy
from joblib import Parallel, delayed, cpu_count
from tqdm.auto import tqdm
import sklearn.pipeline
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
    mean_squared_log_error,
)
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import ParameterSampler
import optuna
from optuna.samplers import TPESampler, RandomSampler

from skforecast.exceptions import LongTrainingWarning
from skforecast.exceptions import IgnoredArgumentWarning
from skforecast.utils import check_backtesting_input
from skforecast.utils import select_n_jobs_backtesting

optuna.logging.set_verbosity(optuna.logging.WARNING) # disable optuna logs

logging.basicConfig(
    format = '%(name)-10s %(levelname)-5s %(message)s', 
    level  = logging.INFO,
)

In [4]:
# Data download
# ==============================================================================
url = (
       'https://raw.githubusercontent.com/JoaquinAmatRodrigo/skforecast/master/'
       'data/simulated_items_sales.csv'
)
data = pd.read_csv(url, sep=',')

# Data preparation
# ==============================================================================
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data = data.set_index('date')
data = data.asfreq('D')
data = data.sort_index()
data.head()

# Split data into train-val-test
# ==============================================================================
end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()

print(
    f"Train dates : {data_train.index.min()} --- {data_train.index.max()}   "
    f"(n={len(data_train)})"
)
print(
    f"Test dates  : {data_test.index.min()} --- {data_test.index.max()}   "
    f"(n={len(data_test)})"
)

Train dates : 2012-01-01 00:00:00 --- 2014-07-15 00:00:00   (n=927)
Test dates  : 2014-07-16 00:00:00 --- 2015-01-01 00:00:00   (n=170)


In [5]:
# Split data into train-val-test
# ==============================================================================
end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()

print(
    f"Train dates : {data_train.index.min()} --- {data_train.index.max()}   "
    f"(n={len(data_train)})"
)
print(
    f"Test dates  : {data_test.index.min()} --- {data_test.index.max()}   "
    f"(n={len(data_test)})"
)

Train dates : 2012-01-01 00:00:00 --- 2014-07-15 00:00:00   (n=927)
Test dates  : 2014-07-16 00:00:00 --- 2015-01-01 00:00:00   (n=170)


## Functions

In [6]:
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries.model_selection_multiseries import _initialize_levels_model_selection_multiseries


def bayesian_search_forecaster(
    forecaster,
    series: pd.DataFrame,
    search_space: Callable,
    steps: int,
    metric: Union[str, Callable, list],
    initial_train_size: int,
    fixed_train_size: bool=True,
    gap: int=0,
    allow_incomplete_fold: bool=True,
    levels: Optional[Union[str, list]]=None,
    exog: Optional[Union[pd.Series, pd.DataFrame]]=None,
    lags_grid: Optional[list]=None,
    refit: Optional[Union[bool, int]]=False,
    n_trials: int=10,
    random_state: int=123,
    return_best: bool=True,
    n_jobs: Optional[Union[int, str]]='auto',
    verbose: bool=True,
    show_progress: bool=True,
    engine: str='optuna',
    kwargs_create_study: dict={},
    kwargs_study_optimize: dict={}
) -> Tuple[pd.DataFrame, object]:
    """
    Bayesian optimization for a Forecaster object using multi-series backtesting 
    and optuna library.
    
    Parameters
    ----------
    forecaster : ForecasterAutoregMultiSeries, ForecasterAutoregMultiSeriesCustom, ForecasterAutoregMultiVariate
        Forecaster model.
    series : pandas DataFrame
        Training time series.
    search_space : Callable
        Function with argument `trial` which returns a dictionary with parameters names 
        (`str`) as keys and Trial object from optuna (trial.suggest_float, 
        trial.suggest_int, trial.suggest_categorical) as values.
    steps : int
        Number of steps to predict.
    metric : str, Callable, list
        Metric used to quantify the goodness of fit of the model.
        
        - If `string`: {'mean_squared_error', 'mean_absolute_error',
        'mean_absolute_percentage_error', 'mean_squared_log_error'}
        - If `Callable`: Function with arguments y_true, y_pred that returns 
        a float.
        - If `list`: List containing multiple strings and/or Callables.
    initial_train_size : int 
        Number of samples in the initial train split.
    fixed_train_size : bool, default `True`
        If True, train size doesn't increase but moves by `steps` in each iteration.
    gap : int, default `0`
        Number of samples to be excluded after the end of each training set and 
        before the test set.
    allow_incomplete_fold : bool, default `True`
        Last fold is allowed to have a smaller number of samples than the 
        `test_size`. If `False`, the last fold is excluded.
    levels : str, list, default `None`
        level (`str`) or levels (`list`) at which the forecaster is optimized. 
        If `None`, all levels are taken into account. The resulting metric will be
        the average of the optimization of all levels.
    exog : pandas Series, pandas DataFrame, default `None`
        Exogenous variable/s included as predictor/s. Must have the same
        number of observations as `y` and should be aligned so that y[i] is
        regressed on exog[i].
    lags_grid : list of int, lists, numpy ndarray or range, default `None`
        Lists of `lags` to try. Only used if forecaster is an instance of 
        `ForecasterAutoregMultiSeries` or `ForecasterAutoregMultiVariate`.
    refit : bool, int, default `False`
        Whether to re-fit the forecaster in each iteration. If `refit` is an integer, 
        the Forecaster will be trained every that number of iterations.
    n_trials : int, default `10`
        Number of parameter settings that are sampled in each lag configuration.
    random_state : int, default `123`
        Sets a seed to the sampling for reproducible output.
    return_best : bool, default `True`
        Refit the `forecaster` using the best found parameters on the whole data.
    n_jobs : int, 'auto', default `'auto'`
        The number of jobs to run in parallel. If `-1`, then the number of jobs is 
        set to the number of cores. If 'auto', `n_jobs` is set using the function
        skforecast.utils.select_n_jobs_backtesting.
        **New in version 0.9.0**
    verbose : bool, default `True`
        Print number of folds used for cv or backtesting.
    show_progress: bool, default `True`
        Whether to show a progress bar.
    engine : str, default `'optuna'`
        Bayesian optimization runs through the optuna library.
    kwargs_create_study : dict, default `{'direction': 'minimize', 'sampler': TPESampler(seed=123)}`
        Keyword arguments (key, value mappings) to pass to optuna.create_study.
    kwargs_study_optimize : dict, default `{}`
        Other keyword arguments (key, value mappings) to pass to study.optimize().

    Returns
    -------
    results : pandas DataFrame
        Results for each combination of parameters.
        - column levels: levels configuration for each iteration.
        - column lags: lags configuration for each iteration.
        - column params: parameters configuration for each iteration.
        - column metric: metric value estimated for each iteration. The resulting 
        metric will be the average of the optimization of all levels.
        - additional n columns with param = value.
    results_opt_best : optuna object
        The best optimization result returned as a FrozenTrial optuna object.
    
    """

    if return_best and exog is not None and (len(exog) != len(series)):
        raise ValueError(
            (f"`exog` must have same number of samples as `series`. "
             f"length `exog`: ({len(exog)}), length `series`: ({len(series)})")
        )

    if engine not in ['optuna']:
        raise ValueError(
            f"`engine` only allows 'optuna', got {engine}."
        )

    results, results_opt_best = _bayesian_search_optuna_multiseries(
                                    forecaster            = forecaster,
                                    series                = series,
                                    exog                  = exog,
                                    levels                = levels, 
                                    lags_grid             = lags_grid,
                                    search_space          = search_space,
                                    steps                 = steps,
                                    metric                = metric,
                                    refit                 = refit,
                                    initial_train_size    = initial_train_size,
                                    fixed_train_size      = fixed_train_size,
                                    gap                   = gap,
                                    allow_incomplete_fold = allow_incomplete_fold,
                                    n_trials              = n_trials,
                                    random_state          = random_state,
                                    return_best           = return_best,
                                    n_jobs                = n_jobs,
                                    verbose               = verbose,
                                    show_progress         = show_progress,
                                    kwargs_create_study   = kwargs_create_study,
                                    kwargs_study_optimize = kwargs_study_optimize
                                )

    return results, results_opt_best


def _bayesian_search_optuna_multiseries(
    forecaster,
    series: pd.DataFrame,
    search_space: Callable,
    steps: int,
    metric: Union[str, Callable, list],
    initial_train_size: int,
    fixed_train_size: bool=True,
    gap: int=0,
    allow_incomplete_fold: bool=True,
    levels: Optional[Union[str, list]]=None,
    exog: Optional[Union[pd.Series, pd.DataFrame]]=None,
    lags_grid: Optional[list]=None,
    refit: Optional[Union[bool, int]]=False,
    n_trials: int=10,
    random_state: int=123,
    return_best: bool=True,
    n_jobs: Optional[Union[int, str]]='auto',
    verbose: bool=True,
    show_progress: bool=True,
    kwargs_create_study: dict={},
    kwargs_study_optimize: dict={}
) -> Tuple[pd.DataFrame, object]:
    """
    Bayesian optimization for a Forecaster object using multi-series backtesting 
    and optuna library.
    
    Parameters
    ----------
    forecaster : ForecasterAutoregMultiSeries, ForecasterAutoregMultiSeriesCustom, ForecasterAutoregMultiVariate
        Forecaster model.
    series : pandas DataFrame
        Training time series.
    search_space : Callable
        Function with argument `trial` which returns a dictionary with parameters names 
        (`str`) as keys and Trial object from optuna (trial.suggest_float, 
        trial.suggest_int, trial.suggest_categorical) as values.
    steps : int
        Number of steps to predict.
    metric : str, Callable, list
        Metric used to quantify the goodness of fit of the model.
        
        - If `string`: {'mean_squared_error', 'mean_absolute_error',
        'mean_absolute_percentage_error', 'mean_squared_log_error'}
        - If `Callable`: Function with arguments y_true, y_pred that returns 
        a float.
        - If `list`: List containing multiple strings and/or Callables.
    initial_train_size : int 
        Number of samples in the initial train split.
    fixed_train_size : bool, default `True`
        If True, train size doesn't increase but moves by `steps` in each iteration.
    gap : int, default `0`
        Number of samples to be excluded after the end of each training set and 
        before the test set.
    allow_incomplete_fold : bool, default `True`
        Last fold is allowed to have a smaller number of samples than the 
        `test_size`. If `False`, the last fold is excluded.
    levels : str, list, default `None`
        level (`str`) or levels (`list`) at which the forecaster is optimized. 
        If `None`, all levels are taken into account. The resulting metric will be
        the average of the optimization of all levels.
    exog : pandas Series, pandas DataFrame, default `None`
        Exogenous variable/s included as predictor/s. Must have the same
        number of observations as `y` and should be aligned so that y[i] is
        regressed on exog[i].
    lags_grid : list of int, lists, numpy ndarray or range, default `None`
        Lists of `lags` to try. Only used if forecaster is an instance of 
        `ForecasterAutoregMultiSeries` or `ForecasterAutoregMultiVariate`.
    refit : bool, int, default `False`
        Whether to re-fit the forecaster in each iteration. If `refit` is an integer, 
        the Forecaster will be trained every that number of iterations.
    n_trials : int, default `10`
        Number of parameter settings that are sampled in each lag configuration.
    random_state : int, default `123`
        Sets a seed to the sampling for reproducible output.
    return_best : bool, default `True`
        Refit the `forecaster` using the best found parameters on the whole data.
    n_jobs : int, 'auto', default `'auto'`
        The number of jobs to run in parallel. If `-1`, then the number of jobs is 
        set to the number of cores. If 'auto', `n_jobs` is set using the function
        skforecast.utils.select_n_jobs_backtesting.
        **New in version 0.9.0**
    verbose : bool, default `True`
        Print number of folds used for cv or backtesting.
    show_progress: bool, default `True`
        Whether to show a progress bar.
    kwargs_create_study : dict, default `{'direction': 'minimize', 'sampler': TPESampler(seed=123)}`
        Keyword arguments (key, value mappings) to pass to optuna.create_study.
    kwargs_study_optimize : dict, default `{}`
        Other keyword arguments (key, value mappings) to pass to study.optimize().

    Returns
    -------
    results : pandas DataFrame
        Results for each combination of parameters.
        - column levels: levels configuration for each iteration.
        - column lags: lags configuration for each iteration.
        - column params: parameters configuration for each iteration.
        - column metric: metric value estimated for each iteration. The resulting 
        metric will be the average of the optimization of all levels.
        - additional n columns with param = value.
    results_opt_best : optuna object
        The best optimization result returned as a FrozenTrial optuna object.

    """
    
    levels = _initialize_levels_model_selection_multiseries(
                 forecaster = forecaster,
                 series     = series,
                 levels     = levels
             )

    if type(forecaster).__name__ == 'ForecasterAutoregMultiSeriesCustom':
        if lags_grid is not None:
            warnings.warn(
                "`lags_grid` ignored if forecaster is an instance of `ForecasterAutoregMultiSeriesCustom`.",
                IgnoredArgumentWarning
            )
        lags_grid = ['custom predictors']
        
    elif lags_grid is None:
        lags_grid = [forecaster.lags]
   
    lags_list = []
    params_list = []
    results_opt_best = None
    if not isinstance(metric, list):
        metric = [metric] 
    metric_dict = {(m if isinstance(m, str) else m.__name__): [] 
                   for m in metric}
    
    if len(metric_dict) != len(metric):
        raise ValueError(
            "When `metric` is a `list`, each metric name must be unique."
        )

    # Objective function using backtesting_forecaster
    def _objective(
        trial,
        search_space          = search_space,
        forecaster            = forecaster,
        series                = series,
        exog                  = exog,
        steps                 = steps,
        levels                = levels,
        metric                = metric,
        initial_train_size    = initial_train_size,
        fixed_train_size      = fixed_train_size,
        gap                   = gap,
        allow_incomplete_fold = allow_incomplete_fold,
        refit                 = refit,
        n_jobs                = n_jobs,
        verbose               = verbose
    ) -> float:
        
        forecaster.set_params(search_space(trial))
        
        metrics_levels = backtesting_forecaster_multiseries(
                             forecaster            = forecaster,
                             series                = series,
                             exog                  = exog,
                             steps                 = steps,
                             levels                = levels,
                             metric                = metric,
                             initial_train_size    = initial_train_size,
                             fixed_train_size      = fixed_train_size,
                             gap                   = gap,
                             allow_incomplete_fold = allow_incomplete_fold,
                             refit                 = refit,
                             n_jobs                = n_jobs,
                             verbose               = verbose,
                             show_progress         = False
                         )[0]
        # Store metrics in the variable metric_values defined outside _objective.
        nonlocal metric_values
        metric_values.append(metrics_levels)

        return metrics_levels.iloc[:, 1].mean()

    print(
        f"""Number of models compared: {n_trials*len(lags_grid)},
         {n_trials} bayesian search in each lag configuration."""
    )

    if show_progress:
        lags_grid = tqdm(lags_grid, desc='lags grid', position=0)

    for lags in lags_grid:
        metric_values = [] # This variable will be modified inside _objective function. 
        # It is a trick to extract multiple values from _objective function since
        # only the optimized value can be returned.

        if type(forecaster).__name__ != 'ForecasterAutoregMultiSeriesCustom':
            forecaster.set_lags(lags)
            lags = forecaster.lags.copy()
        
        if 'sampler' in kwargs_create_study.keys():
            kwargs_create_study['sampler']._rng = np.random.RandomState(random_state)
            kwargs_create_study['sampler']._random_sampler = RandomSampler(seed=random_state)

        study = optuna.create_study(**kwargs_create_study)

        if 'sampler' not in kwargs_create_study.keys():
            study.sampler = TPESampler(seed=random_state)

        study.optimize(_objective, n_trials=n_trials, **kwargs_study_optimize)

        best_trial = study.best_trial

        if search_space(best_trial).keys() != best_trial.params.keys():
            raise ValueError(
                f"""Some of the key values do not match the search_space key names.
                Dict keys     : {list(search_space(best_trial).keys())}
                Trial objects : {list(best_trial.params.keys())}."""
            )
        
        for i, trial in enumerate(study.get_trials()):
            params_list.append(trial.params)
            lags_list.append(lags)

            m_values = metric_values[i]
            for m in metric:
                m_name = m if isinstance(m, str) else m.__name__
                metric_dict[m_name].append(m_values[m_name].mean())
        
        if results_opt_best is None:
            results_opt_best = best_trial
        else:
            if best_trial.value < results_opt_best.value:
                results_opt_best = best_trial

    results = pd.DataFrame({
                  'levels': [levels]*len(lags_list),
                  'lags'  : lags_list,
                  'params': params_list,
                  **metric_dict
              })

    results = results.sort_values(by=list(metric_dict.keys())[0], ascending=True)
    results = pd.concat([results, results['params'].apply(pd.Series)], axis=1)
    
    if return_best:
        
        best_lags = results['lags'].iloc[0]
        best_params = results['params'].iloc[0]
        best_metric = results[list(metric_dict.keys())[0]].iloc[0]
        
        if type(forecaster).__name__ != 'ForecasterAutoregMultiSeriesCustom':
            forecaster.set_lags(best_lags)
        forecaster.set_params(best_params)
        forecaster.fit(series=series, exog=exog, store_in_sample_residuals=True)
        
        print(
            f"`Forecaster` refitted using the best-found lags and parameters, "
            f"and the whole data set: \n"
            f"  Lags: {best_lags} \n"
            f"  Parameters: {best_params}\n"
            f"  Backtesting metric: {best_metric}\n"
            f"  Levels: {results['levels'].iloc[0]}\n"
        )
            
    return results, results_opt_best

## Tests

In [9]:
from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import series

series.head(3)

,l1,l2
0,0.696469,0.120629
1,0.286139,0.826341
2,0.226851,0.603060


In [10]:
forecaster = ForecasterAutoregMultiSeries(
                    regressor = RandomForestRegressor(random_state=123),
                    lags      = 2 # Placeholder, the value will be overwritten
                 )

steps = 3
n_validation = 12
lags_grid = [2, 4]

def search_space(trial):
    search_space  = {'n_estimators'    : trial.suggest_int('n_estimators', 10, 20),
                        'min_samples_leaf': trial.suggest_float('min_samples_leaf', 0.1, 1., log=True),
                        'max_features'    : trial.suggest_categorical('max_features', ['log2', 'sqrt'])}
    
    return search_space

results = _bayesian_search_optuna_multiseries(
                forecaster         = forecaster,
                series             = series,
                lags_grid          = lags_grid,
                search_space       = search_space,
                steps              = steps,
                metric             = 'mean_absolute_error',
                refit              = True,
                initial_train_size = len(series) - n_validation,
                fixed_train_size   = True,
                n_trials           = 10,
                random_state       = 123,
                return_best        = False,
                verbose            = False
            )[0]

results

Number of models compared: 20,
         10 bayesian search in each lag configuration.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

,levels,lags,params,mean_absolute_error,n_estimators,min_samples_leaf,max_features
4,"[l1, l2]","[1, 2]","{'n_estimators': 12, 'min_samples_leaf': 0.149...",0.209610,12,0.149779,sqrt
2,"[l1, l2]","[1, 2]","{'n_estimators': 15, 'min_samples_leaf': 0.246...",0.209730,15,0.246671,sqrt
6,"[l1, l2]","[1, 2]","{'n_estimators': 17, 'min_samples_leaf': 0.210...",0.211587,17,0.210358,log2
0,"[l1, l2]","[1, 2]","{'n_estimators': 17, 'min_samples_leaf': 0.193...",0.212235,17,0.193259,sqrt
8,"[l1, l2]","[1, 2]","{'n_estimators': 14, 'min_samples_leaf': 0.311...",0.212961,14,0.311663,log2
3,"[l1, l2]","[1, 2]","{'n_estimators': 14, 'min_samples_leaf': 0.114...",0.213406,14,0.114730,sqrt
1,"[l1, l2]","[1, 2]","{'n_estimators': 17, 'min_samples_leaf': 0.264...",0.214647,17,0.264915,log2
12,"[l1, l2]","[1, 2, 3, 4]","{'n_estimators': 15, 'min_samples_leaf': 0.246...",0.215478,15,0.246671,sqrt
19,"[l1, l2]","[1, 2, 3, 4]","{'n_estimators': 14, 'min_samples_leaf': 0.782...",0.215708,14,0.782329,log2
17,"[l1, l2]","[1, 2, 3, 4]","{'n_estimators': 13, 'min_samples_leaf': 0.427...",0.215743,13,0.427539,sqrt


In [21]:
[list(x) for x in results['lags'].to_numpy()]

[[1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4]]

In [22]:
results['params'].to_numpy()

array([{'n_estimators': 12, 'min_samples_leaf': 0.14977928606210794, 'max_features': 'sqrt'},
       {'n_estimators': 15, 'min_samples_leaf': 0.2466706727024324, 'max_features': 'sqrt'},
       {'n_estimators': 17, 'min_samples_leaf': 0.21035794225904136, 'max_features': 'log2'},
       {'n_estimators': 17, 'min_samples_leaf': 0.19325882509735576, 'max_features': 'sqrt'},
       {'n_estimators': 14, 'min_samples_leaf': 0.3116628929828935, 'max_features': 'log2'},
       {'n_estimators': 14, 'min_samples_leaf': 0.1147302385573586, 'max_features': 'sqrt'},
       {'n_estimators': 17, 'min_samples_leaf': 0.2649149454284987, 'max_features': 'log2'},
       {'n_estimators': 15, 'min_samples_leaf': 0.2466706727024324, 'max_features': 'sqrt'},
       {'n_estimators': 14, 'min_samples_leaf': 0.782328520465639, 'max_features': 'log2'},
       {'n_estimators': 13, 'min_samples_leaf': 0.42753938073418213, 'max_features': 'sqrt'},
       {'n_estimators': 16, 'min_samples_leaf': 0.707020154488976, 

In [23]:
results['mean_absolute_error'].to_numpy()

array([0.20961049, 0.20973005, 0.21158737, 0.21223533, 0.21296057,
       0.21340636, 0.21464693, 0.2154781 , 0.21570766, 0.2157434 ,
       0.21616556, 0.21634926, 0.2165384 , 0.21665671, 0.21684708,
       0.21706527, 0.21732157, 0.21839139, 0.21990283, 0.22493842])

In [24]:
results['n_estimators'].to_numpy()

array([12, 15, 17, 17, 14, 14, 17, 15, 14, 13, 16, 16, 14, 13, 17, 17, 17,
       14, 12, 14], dtype=int64)

In [25]:
results['min_samples_leaf'].to_numpy()

array([0.14977929, 0.24667067, 0.21035794, 0.19325883, 0.31166289,
       0.11473024, 0.26491495, 0.24667067, 0.78232852, 0.42753938,
       0.70702015, 0.70702015, 0.78232852, 0.42753938, 0.26491495,
       0.19325883, 0.21035794, 0.31166289, 0.14977929, 0.11473024])

In [26]:
results['max_features'].to_numpy()

array(['sqrt', 'sqrt', 'log2', 'sqrt', 'log2', 'sqrt', 'log2', 'sqrt',
       'log2', 'sqrt', 'log2', 'log2', 'log2', 'sqrt', 'log2', 'sqrt',
       'log2', 'log2', 'sqrt', 'sqrt'], dtype=object)

In [27]:
results.index

Index([4, 2, 6, 0, 8, 3, 1, 12, 19, 17, 15, 5, 9, 7, 11, 10, 16, 18, 14, 13], dtype='int64')

In [30]:
forecaster = ForecasterAutoregMultiSeries(
                    regressor = RandomForestRegressor(random_state=123),
                    lags      = 2 # Placeholder, the value will be overwritten
                 )

steps = 3
n_validation = 12
lags_grid = [2, 4]

def search_space(trial):
    search_space  = {'n_estimators'    : trial.suggest_int('n_estimators', 10, 20),
                        'min_samples_leaf': trial.suggest_float('min_samples_leaf', 0.1, 1., log=True),
                        'max_features'    : trial.suggest_categorical('max_features', ['log2', 'sqrt'])}
    
    return search_space

results = _bayesian_search_optuna_multiseries(
                forecaster         = forecaster,
                series             = series,
                lags_grid          = lags_grid,
                search_space       = search_space,
                steps              = steps,
                metric             = 'mean_absolute_error',
                refit              = True,
                initial_train_size = len(series) - n_validation,
                fixed_train_size   = True,
                n_trials           = 10,
                random_state       = 123,
                return_best        = False,
                verbose            = False
            )[0]

expected_results = pd.DataFrame({
    'levels': [['l1', 'l2']]*2*10,
    'lags'  :[[1, 2], [1, 2], [1, 2], [1, 2], [1, 2],
                [1, 2], [1, 2], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4],
                [1, 2, 3, 4], [1, 2], [1, 2], [1, 2], [1, 2, 3, 4],
                [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4]],
    'params':[{'n_estimators': 12, 'min_samples_leaf': 0.14977928606210794, 'max_features': 'sqrt'},
                {'n_estimators': 15, 'min_samples_leaf': 0.2466706727024324, 'max_features': 'sqrt'},
                {'n_estimators': 17, 'min_samples_leaf': 0.21035794225904136, 'max_features': 'log2'},
                {'n_estimators': 17, 'min_samples_leaf': 0.19325882509735576, 'max_features': 'sqrt'},
                {'n_estimators': 14, 'min_samples_leaf': 0.3116628929828935, 'max_features': 'log2'},
                {'n_estimators': 14, 'min_samples_leaf': 0.1147302385573586, 'max_features': 'sqrt'},
                {'n_estimators': 17, 'min_samples_leaf': 0.2649149454284987, 'max_features': 'log2'},
                {'n_estimators': 15, 'min_samples_leaf': 0.2466706727024324, 'max_features': 'sqrt'},
                {'n_estimators': 14, 'min_samples_leaf': 0.782328520465639, 'max_features': 'log2'},
                {'n_estimators': 13, 'min_samples_leaf': 0.42753938073418213, 'max_features': 'sqrt'},
                {'n_estimators': 16, 'min_samples_leaf': 0.707020154488976, 'max_features': 'log2'},
                {'n_estimators': 16, 'min_samples_leaf': 0.707020154488976, 'max_features': 'log2'},
                {'n_estimators': 14, 'min_samples_leaf': 0.782328520465639, 'max_features': 'log2'},
                {'n_estimators': 13, 'min_samples_leaf': 0.42753938073418213, 'max_features': 'sqrt'},
                {'n_estimators': 17, 'min_samples_leaf': 0.2649149454284987, 'max_features': 'log2'},
                {'n_estimators': 17, 'min_samples_leaf': 0.19325882509735576, 'max_features': 'sqrt'},
                {'n_estimators': 17, 'min_samples_leaf': 0.21035794225904136, 'max_features': 'log2'},
                {'n_estimators': 14, 'min_samples_leaf': 0.3116628929828935, 'max_features': 'log2'},
                {'n_estimators': 12, 'min_samples_leaf': 0.14977928606210794, 'max_features': 'sqrt'},
                {'n_estimators': 14, 'min_samples_leaf': 0.1147302385573586, 'max_features': 'sqrt'}],
    'mean_absolute_error':np.array([0.20961049, 0.20973005, 0.21158737, 0.21223533, 0.21296057,
                                    0.21340636, 0.21464693, 0.2154781 , 0.21570766, 0.2157434 ,
                                    0.21616556, 0.21634926, 0.2165384 , 0.21665671, 0.21684708,
                                    0.21706527, 0.21732157, 0.21839139, 0.21990283, 0.22493842]),                                                               
    'n_estimators' :np.array([12, 15, 17, 17, 14, 14, 17, 15, 14, 13, 16, 16, 14, 13, 17, 17, 17, 14, 12, 14]),
    'min_samples_leaf' :np.array([0.14977929, 0.24667067, 0.21035794, 0.19325883, 0.31166289,
                                    0.11473024, 0.26491495, 0.24667067, 0.78232852, 0.42753938,
                                    0.70702015, 0.70702015, 0.78232852, 0.42753938, 0.26491495,
                                    0.19325883, 0.21035794, 0.31166289, 0.14977929, 0.11473024]),
    'max_features' :['sqrt', 'sqrt', 'log2', 'sqrt', 'log2', 'sqrt', 'log2', 'sqrt',
                        'log2', 'sqrt', 'log2', 'log2', 'log2', 'sqrt', 'log2', 'sqrt',
                        'log2', 'log2', 'sqrt', 'sqrt']
    },
    index=pd.Index([4, 2, 6, 0, 8, 3, 1, 12, 19, 17, 15, 5, 9, 7, 11, 10, 16, 18, 14, 13], dtype="int64")
).sort_values(by='mean_absolute_error', ascending=True)

pd.testing.assert_frame_equal(results, expected_results, check_dtype=False)

Number of models compared: 20,
         10 bayesian search in each lag configuration.


lags grid:   0%|          | 0/2 [00:00<?, ?it/s]

In [47]:
forecaster = ForecasterAutoregMultiSeries(
                regressor = Ridge(random_state=123),
                lags      = 4
             )

steps              = 3
metric             = 'mean_absolute_error'
levels             = ['l1']
n_validation       = 12
initial_train_size = len(series) - n_validation
fixed_train_size   = True
refit              = True
verbose            = False
show_progress      = False

n_trials = 10
random_state = 123

def objective(
    trial,
    forecaster         = forecaster,
    series             = series,
    levels             = levels,
    steps              = steps,
    metric             = metric,
    initial_train_size = initial_train_size,
    fixed_train_size   = fixed_train_size,
    refit              = refit,
    verbose            = verbose,
    show_progress      = show_progress
) -> float:
    
    alpha = trial.suggest_float('alpha', 1e-2, 1.0)
    
    forecaster = ForecasterAutoregMultiSeries(
                    regressor = Ridge(random_state=random_state, 
                                        alpha=alpha),
                    lags      = 4
                    )

    metrics_levels, _ = backtesting_forecaster_multiseries(
                            forecaster         = forecaster,
                            series             = series,
                            levels             = levels,
                            steps              = steps,
                            metric             = metric,
                            initial_train_size = initial_train_size,
                            fixed_train_size   = fixed_train_size,
                            refit              = refit,
                            verbose            = verbose,
                            show_progress      = show_progress     
                        )

    return abs(metrics_levels.iloc[:, 1].mean())

study = optuna.create_study(direction="minimize", 
                            sampler=TPESampler(seed=random_state))
study.optimize(objective, n_trials=n_trials)

best_trial = study.best_trial

lags_grid = [2, 4]
def search_space(trial):
    search_space  = {'alpha': trial.suggest_float('alpha', 1e-2, 1.0)}

    return search_space

return_best  = False

_, results_opt_best = _bayesian_search_optuna_multiseries(
                        forecaster         = forecaster,
                        series             = series,
                        levels             = levels, 
                        lags_grid          = lags_grid,
                        search_space       = search_space,
                        steps              = steps,
                        metric             = metric,
                        refit              = refit,
                        initial_train_size = initial_train_size,
                        fixed_train_size   = fixed_train_size,
                        n_trials           = n_trials,
                        return_best        = return_best,
                        verbose            = verbose,
                        show_progress      = show_progress
                    )

assert best_trial.number == results_opt_best.number
assert best_trial.values == results_opt_best.values
assert best_trial.params == results_opt_best.params

Number of models compared: 20,
         10 bayesian search in each lag configuration.


In [48]:
_

,levels,lags,params,mean_absolute_error,alpha
12,[l1],"[1, 2, 3, 4]",{'alpha': 0.2345829390285611},0.215850,0.234583
11,[l1],"[1, 2, 3, 4]",{'alpha': 0.29327794160087567},0.215857,0.293278
19,[l1],"[1, 2, 3, 4]",{'alpha': 0.398196343012209},0.215871,0.398196
15,[l1],"[1, 2, 3, 4]",{'alpha': 0.42887539552321635},0.215874,0.428875
18,[l1],"[1, 2, 3, 4]",{'alpha': 0.48612258246951734},0.215882,0.486123
13,[l1],"[1, 2, 3, 4]",{'alpha': 0.5558016213920624},0.215890,0.555802
17,[l1],"[1, 2, 3, 4]",{'alpha': 0.6879814411990146},0.215907,0.687981
10,[l1],"[1, 2, 3, 4]",{'alpha': 0.6995044937418831},0.215908,0.699504
14,[l1],"[1, 2, 3, 4]",{'alpha': 0.7222742800877074},0.215911,0.722274
16,[l1],"[1, 2, 3, 4]",{'alpha': 0.9809565564007693},0.215942,0.980957


In [49]:
[list(x) for x in _['lags'].to_numpy()]

[[1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2],
 [1, 2]]

In [52]:
_['params'].to_numpy()

array([{'alpha': 0.2345829390285611}, {'alpha': 0.29327794160087567},
       {'alpha': 0.398196343012209}, {'alpha': 0.42887539552321635},
       {'alpha': 0.48612258246951734}, {'alpha': 0.5558016213920624},
       {'alpha': 0.6879814411990146}, {'alpha': 0.6995044937418831},
       {'alpha': 0.7222742800877074}, {'alpha': 0.9809565564007693},
       {'alpha': 0.2345829390285611}, {'alpha': 0.29327794160087567},
       {'alpha': 0.398196343012209}, {'alpha': 0.42887539552321635},
       {'alpha': 0.48612258246951734}, {'alpha': 0.5558016213920624},
       {'alpha': 0.6879814411990146}, {'alpha': 0.6995044937418831},
       {'alpha': 0.7222742800877074}, {'alpha': 0.9809565564007693}],
      dtype=object)

In [54]:
_['mean_absolute_error'].to_numpy()

array([0.21584992, 0.21585737, 0.2158706 , 0.21587445, 0.2158816 ,
       0.21589025, 0.21590653, 0.21590794, 0.21591073, 0.21594197,
       0.2163035 , 0.21630557, 0.21630925, 0.21631032, 0.21631231,
       0.21631472, 0.21631927, 0.21631967, 0.21632045, 0.21632921])

In [55]:
_['alpha'].to_numpy()

array([0.23458294, 0.29327794, 0.39819634, 0.4288754 , 0.48612258,
       0.55580162, 0.68798144, 0.69950449, 0.72227428, 0.98095656,
       0.23458294, 0.29327794, 0.39819634, 0.4288754 , 0.48612258,
       0.55580162, 0.68798144, 0.69950449, 0.72227428, 0.98095656])

In [53]:
_.index

Index([12, 11, 19, 15, 18, 13, 17, 10, 14, 16, 2, 1, 9, 5, 8, 3, 7, 0, 4, 6], dtype='int64')